# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import sagemaker
from sagemaker import get_execution_role
import subprocess
import json





# magic word for producing visualizations in notebook
%matplotlib inline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Setup Sagemaker Session
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
execution_role = sagemaker.session.get_execution_role()
region = sagemaker_session.boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
#download data to notebook
#define data location constants
local_data_dir = 'data'
s3_data_path = f's3://{bucket}/data' 
s3_model_path = f's3://{bucket}/model'

## Initial Model and Kaggle Submission

Below I will be setting up the an initial AutoGluon run without any refienment of the data. Then I'll be submitting to Kaggle.

In [4]:
%%capture

!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir
!pip install kaggle
!pip install python-dotenv
from autogluon.tabular import TabularPredictor


### Setting up Kaggle Creds


In [12]:
!mkdir -p kaggle
%env KAGGLE_CONFIG_DIR=kaggle
!touch kaggle/kaggle.json
!chmod 600 kaggle/kaggle.json

env: KAGGLE_CONFIG_DIR=kaggle


In [21]:
from dotenv import dotenv_values

CONFIG = dotenv_values('env.txt')
kaggle_username = CONFIG['KAGGLE_USERNAME']
kaggle_key = CONFIG['KAGGLE_KEY']

# Save API token the kaggle.json file
with open("kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

### Downloading and Prepping Data

In [5]:
train_data = pd.read_csv(f'{s3_data_path}/train.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/tmp/ipykernel_7972/3792393069.py:1: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(f'{s3_data_path}/train.csv')


In [6]:
train_data.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1763,2,1.0,8.0,NaN,NaN,NaN,NaN,8.0,15.0,...,5.0,2.0,1.0,6.0,9.0,3.0,3,0,2,4
1,1771,1,4.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,1.0,4.0,9.0,7.0,1,0,2,3
2,1776,1,1.0,9.0,NaN,NaN,NaN,NaN,7.0,0.0,...,6.0,4.0,2.0,NaN,9.0,2.0,3,0,1,4
3,1460,2,1.0,6.0,NaN,NaN,NaN,NaN,6.0,4.0,...,8.0,11.0,11.0,6.0,9.0,1.0,3,0,2,4
4,1783,2,1.0,9.0,NaN,NaN,NaN,NaN,9.0,53.0,...,2.0,2.0,1.0,6.0,9.0,3.0,3,0,1,3


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42962 entries, 0 to 42961
Columns: 367 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(6)
memory usage: 120.3+ MB


### Doing Simple Training
To start for our first submission we're going to drop all the columns but the ones explored in the proposal section, this is simply to get a basic submission and baseline to compare against at the end of the project.

In [6]:
train_data[['RESPONSE', 'LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']].describe()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42962.000000,42962.000000,42357.000000,42357.000000,42357.000000,42962.000000
mean,0.012383,42803.120129,17.661071,5.274996,5.927001,1.595084
std,0.110589,24778.339984,14.085702,4.470538,3.398336,0.490881
min,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,0.000000,21284.250000,6.000000,2.000000,3.000000,1.000000
50%,0.000000,42710.000000,15.000000,4.000000,5.000000,2.000000
75%,0.000000,64340.500000,32.000000,10.000000,9.000000,2.000000
max,1.000000,85795.000000,40.000000,12.000000,10.000000,2.000000


In [6]:
selected_columns = ['RESPONSE', 'LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']
init_train = train_data[selected_columns]

In [9]:
init_train.head()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
0,0,1763,8.0,2.0,3.0,2
1,0,1771,19.0,5.0,9.0,2
2,0,1776,0.0,0.0,10.0,1
3,0,1460,16.0,4.0,3.0,2
4,0,1783,9.0,3.0,6.0,1


In [8]:
init_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42962 entries, 0 to 42961
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RESPONSE             42962 non-null  int64  
 1   LNR                  42962 non-null  int64  
 2   LP_LEBENSPHASE_FEIN  42357 non-null  float64
 3   LP_LEBENSPHASE_GROB  42357 non-null  float64
 4   LP_STATUS_FEIN       42357 non-null  float64
 5   ANREDE_KZ            42962 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 2.0 MB


In [7]:
features = ['LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']
init_train.dropna(subset=features, inplace=True)

/tmp/ipykernel_7972/3271910578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_train.dropna(subset=features, inplace=True)


In [ ]:
init_train.info()

In [18]:
init_train.set_index('LNR', inplace=True)
predictor = TabularPredictor(label="RESPONSE", eval_metric='log_loss').fit(
    train_data=init_train,
    time_limit=600,
    presets="best_quality"
)

2023-11-29 14:39:33,574	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.


In [19]:
model_path = predictor.path
print(f"The model was saved in: {model_path}")

The model was saved in: AutogluonModels/ag-20231129_143013/


In [9]:
## Make copy of test file for submission
test_data = pd.read_csv(f'{s3_data_path}/test.csv')

/tmp/ipykernel_7972/9532272.py:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(f'{s3_data_path}/test.csv')


In [10]:
init_test_columns = [col for col in selected_columns if col != 'RESPONSE']
init_test = test_data[init_test_columns]
init_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42833 entries, 0 to 42832
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LNR                  42833 non-null  int64  
 1   LP_LEBENSPHASE_FEIN  42255 non-null  float64
 2   LP_LEBENSPHASE_GROB  42255 non-null  float64
 3   LP_STATUS_FEIN       42255 non-null  float64
 4   ANREDE_KZ            42833 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


In [11]:
init_test.describe()

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42833.000000,42255.000000,42255.000000,42255.000000,42833.000000
mean,42993.165620,17.775175,5.313194,5.936528,1.595475
std,24755.599728,14.096616,4.475535,3.412463,0.490806
min,2.000000,0.000000,0.000000,1.000000,1.000000
25%,21650.000000,6.000000,2.000000,3.000000,1.000000
50%,43054.000000,15.000000,4.000000,5.000000,2.000000
75%,64352.000000,32.000000,10.000000,9.000000,2.000000
max,85794.000000,40.000000,12.000000,10.000000,2.000000


In [16]:
## Need to fill in NA values
init_test['LP_LEBENSPHASE_FEIN'].fillna(init_test['LP_LEBENSPHASE_FEIN'].mean(), inplace=True)
init_test['LP_LEBENSPHASE_GROB'].fillna(init_test['LP_LEBENSPHASE_GROB'].mean(), inplace=True)
init_test['LP_STATUS_FEIN'].fillna(init_test['LP_STATUS_FEIN'].mean(), inplace=True)
init_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42833 entries, 0 to 42832
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LNR                  42833 non-null  int64  
 1   LP_LEBENSPHASE_FEIN  42833 non-null  float64
 2   LP_LEBENSPHASE_GROB  42833 non-null  float64
 3   LP_STATUS_FEIN       42833 non-null  float64
 4   ANREDE_KZ            42833 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


/tmp/ipykernel_7972/2282792149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test['LP_LEBENSPHASE_FEIN'].fillna(init_test['LP_LEBENSPHASE_FEIN'].mean(), inplace=True)
/tmp/ipykernel_7972/2282792149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test['LP_LEBENSPHASE_GROB'].fillna(init_test['LP_LEBENSPHASE_GROB'].mean(), inplace=True)
/tmp/ipykernel_7972/2282792149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42833.000000,42833.000000,42833.000000,42833.000000,42833.000000
mean,42993.165620,17.775175,5.313194,5.936528,1.595475
std,24755.599728,14.001179,4.445235,3.389360,0.490806
min,2.000000,0.000000,0.000000,1.000000,1.000000
25%,21650.000000,6.000000,2.000000,3.000000,1.000000
50%,43054.000000,15.000000,4.000000,5.936528,2.000000
75%,64352.000000,32.000000,10.000000,9.000000,2.000000
max,85794.000000,40.000000,12.000000,10.000000,2.000000


In [17]:
init_test.describe()

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42833.000000,42833.000000,42833.000000,42833.000000,42833.000000
mean,42993.165620,17.775175,5.313194,5.936528,1.595475
std,24755.599728,14.001179,4.445235,3.389360,0.490806
min,2.000000,0.000000,0.000000,1.000000,1.000000
25%,21650.000000,6.000000,2.000000,3.000000,1.000000
50%,43054.000000,15.000000,4.000000,5.936528,2.000000
75%,64352.000000,32.000000,10.000000,9.000000,2.000000
max,85794.000000,40.000000,12.000000,10.000000,2.000000


In [20]:
predictions = predictor.predict(init_test)
predictions.head()

0    0
1    0
2    0
3    0
4    0
Name: RESPONSE, dtype: int64

In [21]:
predictions.describe()

count    42833.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: RESPONSE, dtype: float64